In [12]:
import gc
import multiprocessing
from typing import Tuple, Any
!pip install pytorch-lightning

In [13]:
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [14]:
from src.tools import *

In [15]:
torch.set_printoptions(precision=3, edgeitems=20, linewidth=250)
torch.set_float32_matmul_precision('high')
np.set_printoptions(precision=3, suppress=True, edgeitems=20, linewidth=250)
INT_BITS = 16
INT_MAX = (1 << (INT_BITS - 1)) - 1
INT_MIN = -(1 << (INT_BITS - 1))

In [16]:
class SumNaturalBinModule(nn.Module):
    def __init__(self, layer_sizes=(INT_BITS, 64, 64, INT_BITS), activation_fun=nn.ReLU()):
        super().__init__()

        self.run_counter = 0

        layers_list = []
        for i in range(len(layer_sizes) - 1):
            layers_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            # if i != len(layer_sizes) - 2:
            layers_list.append(activation_fun)

        self.l1 = nn.Sequential(*layers_list)

        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        self.run_counter += 1
        return self.l1(x.double())


class SumNaturalBinAutoEncoder(pl.LightningModule):
    def __init__(self, encoder: SumNaturalBinModule, loss_function=F.mse_loss, lr=3 * 1e-3):
        super().__init__()
        self.encoder = encoder
        self.loss_function = loss_function
        self.lr = lr

    def predict(self, x):
        return self.encoder(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        loss = self.loss_function(y_hat, y)
        return loss

    @staticmethod
    def _get_accuracy(y_hat, y) -> tuple[float, float]:
        eq = (y == torch.round(y_hat))
        return eq.all(axis=1).double().mean(), eq.double().mean()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        loss = self.loss_function(y_hat, y)
        acc, bin_acc = self._get_accuracy(y_hat, y)
        metrics = {"test_acc": acc, "test_loss": loss, "test_bin_acc": bin_acc}
        self.log_dict(metrics)
        return metrics

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2.5 * 1e-3)
        return optimizer

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        val_loss = self.loss_function(y_hat, y)
        acc, bin_acc = self._get_accuracy(y_hat, y)
        metrics = {"test_acc": acc, "test_loss": val_loss, "test_bin_acc": bin_acc}
        self.log_dict(metrics)
        return metrics

        # Return the validation loss and any other metrics you computed
        # return {'val_loss': val_loss, 'val_accuracy': accuracy}

In [17]:
from torch.utils.data import Dataset
import torch
import os
from tqdm import tqdm
import torch.random
import numpy as np
import pandas as pd

In [18]:
import warnings


class SimpleRandomNaturalBinSumDataset(Dataset):
    def __init__(self, size, transform=None, target_transform=None):
        self.size = size
        self.transform = transform
        self.target_transform = target_transform
        ints1 = np.random.randint(INT_MIN, INT_MAX, size=self.size, dtype=np.int32)
        ints2 = np.random.randint(INT_MIN, INT_MAX, size=self.size, dtype=np.int32)

        array1 = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in tqdm(ints1)])
        array2 = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in tqdm(ints2)])
        self.X = np.concatenate((array1.reshape((self.size, INT_BITS)),
                                 array2.reshape((self.size, INT_BITS))), axis=1)

        self.labels = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in
                                tqdm((ints1 + ints2) % (1 << INT_BITS) + INT_MIN)])

    @staticmethod
    def compose_from_bool_array(bool_array):
        if not isinstance(bool_array, np.ndarray):
            raise ValueError("bool_array should be a numpy ndarray.")

        if bool_array.shape != (INT_BITS,):
            raise ValueError("bool_array should be a 1-dimensional array of size 32.")

        binary_str = "".join(str(int(bit)) for bit in bool_array)
        first_bit = binary_str[0]
        number_str = binary_str[1:]

        if first_bit == '0':
            return int(number_str, 2)
        else:
            return int(number_str, 2) + INT_MIN

    @staticmethod
    def decompose_to_bool_array(number):
        if not isinstance(number, np.int32) and not isinstance(number, np.int64) and not isinstance(number, int):
            raise ValueError(f"Number should be an integer. Received {type(number)}")

        if number < 0:
            return SimpleRandomNaturalBinSumDataset._decompose_positive_add_first_bit(number - INT_MIN, str(1))
        else:
            return SimpleRandomNaturalBinSumDataset._decompose_positive_add_first_bit(number, str(0))

    @staticmethod
    def _decompose_positive_add_first_bit(number, first_bit: str):
        binary_str = first_bit + bin(number)[2:].zfill(INT_BITS - 1)
        bool_array = np.array([int(bit) for bit in binary_str], dtype=np.float64)
        return bool_array

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        values, labels = self.X[idx], self.labels[idx]

        if self.transform:
            values = self.transform(values)
        if self.target_transform:
            labels = self.target_transform(labels)
        return values, labels


In [19]:
class PyplineOfSunModule:
    def __init__(self, model: SumNaturalBinAutoEncoder):
        self.model = model

    def predict(self, x: int, y: int):
        expected = x + y
        x_arr = SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(x)
        y_arr = SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(y)
        model_input = torch.tensor(np.concatenate((x_arr, y_arr)))
        predictions = self.model.predict(model_input).detach().numpy().round()
        print(f"Prediction:\t{predictions}")
        print(f"Expected:\t{SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(expected)}")
        return SimpleRandomNaturalBinSumDataset.compose_from_bool_array(predictions)


In [20]:
# pypline_of_sum = PyplineOfSunModule(torch.load(f"autoencoder-V03.pt"))
# pypline_of_sum.predict(np.int32(-3), np.int32(5))

In [21]:
# Parameters:
activation_fun = nn.Sigmoid()
# layer_sizes = [INT_BITS, 8, 8, 8, INT_BITS]
# layer_sizes = [INT_BITS * 2, 512, 128, INT_BITS]
# layer_sizes = [INT_BITS * 2, 128, INT_BITS]
layer_sizes = [INT_BITS * 2, 128, 64, INT_BITS]
# layer_sizes = [INT_BITS * 2, 128, INT_BITS]
# layer_sizes = [1, 32, 32, 32, 1]

In [35]:
dataset = SimpleRandomNaturalBinSumDataset(1_000_000)
train_loader = DataLoader(dataset, batch_size=10_000,
                          num_workers=multiprocessing.cpu_count(),
                          )
valid_dataset = SimpleRandomNaturalBinSumDataset(100_000)
valid_loader = DataLoader(valid_dataset, batch_size=5_000,
                          num_workers=multiprocessing.cpu_count(),
                          )

100%|██████████| 100000/100000 [00:00<00:00, 167688.11it/s]


In [23]:
# model
autoencoder = SumNaturalBinAutoEncoder(SumNaturalBinModule(layer_sizes, activation_fun), lr=0.01)

In [36]:
test_dataset = SimpleRandomNaturalBinSumDataset(500_000)
test_loader = DataLoader(test_dataset, batch_size=1000, num_workers=multiprocessing.cpu_count())

100%|██████████| 500000/500000 [00:02<00:00, 174527.14it/s]


In [25]:
# train model
trainer = pl.Trainer(max_epochs=30)

trainer.fit(autoencoder, train_dataloaders=train_loader)
trainer.test(autoencoder, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/andrey/.local/share/virtualenvs/ML-H-spring_semesester_project-nJO6SWcT/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/home/andrey/.local/share/virtualenvs/ML-H-spring_semesester_project-nJO6SWcT/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but h

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                 0.000134
      test_bin_acc               0.6150815
        test_loss           0.1990419214881198
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.000134,
  'test_loss': 0.1990419214881198,
  'test_bin_acc': 0.6150815}]

In [ ]:
autoencoder.lr = 0.001
# for i in range(5):
# print("#" * 40 + " " * 10 + f"Iteration № {i:5}" + " " * 10 + "#" * 40)
trainer = pl.Trainer(max_epochs=30)
trainer.fit(autoencoder, train_dataloaders=train_loader)
trainer.test(autoencoder, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                | Params
------------------------------------------------
0 | encoder | SumNaturalBinModule | 13.5 K
------------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [43]:
torch.save(autoencoder, f"autoencoder-16-BIT-[128-64]-V02.pt")

In [27]:
autoencoder.encoder

SumNaturalBinModule(
  (l1): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=64, out_features=16, bias=True)
    (5): Sigmoid()
  )
)

In [29]:
# x_rand = torch.tensor(np.random.randint(low=0, high=2, size=(1, INT_BITS)).astype(np.float64))
# y_hat = autoencoder.predict(x_rand).round()
# print(x_rand == y_hat)

In [30]:
# import plotly.express as px

In [31]:
# print(x[0].astype(int))
# print(y[0])
# print(y[0].round().astype(int))

In [32]:
# print(x_to_plot)
# print(y_to_plot)

In [33]:
# fig = px.line(x=x_to_plot, y=y_to_plot)
# fig.show()
# 
# fig = px.line(x=x_to_plot, y=differ.sum(axis=1))
# fig.show()